## Dataset statistics

In [ ]:



from func import *
import os
import pandas as pd
import numpy as np


def make_sens_vector_(df, dataset, sens_dict):
    if dataset in ['Walmart-Amazon', 'Fodors-Zagat', 'COMPAS']:
        df['left_contains_s'] = df[sens_dict[dataset][0]] == sens_dict[dataset][1]
        
    elif dataset in ['Beer', 'Amazon-Google', 'iTunes-Amazon', 'DBLP-GoogleScholar']:
        df[sens_dict[dataset][0]] = df[sens_dict[dataset][0]].astype(str)
        df['left_contains_s'] = df[sens_dict[dataset][0]].str.lower().str.contains(sens_dict[dataset][1].lower())
    else:
        df[sens_dict[dataset][0]] = df[sens_dict[dataset][0]].astype(str)
        df['left_contains_s'] = df[sens_dict[dataset][0]].apply(lambda x: x.replace('&#216;', '').replace('&#214;', '').replace('&#237;', ',').split(',')[-1].strip())


        sens = [] 
        for x  in list(df['left_contains_s']):
            try: 
                tmp = str(gender_rev(x))
            except:
                tmp = '?'
            if 'female' in tmp:
                GENDER = True
            else:
                GENDER = False
            sens.append(GENDER)

        df['left_contains_s'] = sens
        result_vector = np.array(df['left_contains_s'])
        sens_attr = np.array(result_vector).reshape(-1)
        return sens_attr

    result_vector = np.array(df['left_contains_s']).astype(int)
    sens_attr = np.array(result_vector).reshape(-1)

    return sens_attr



for path in os.listdir('data/'):
    if 'DS_Store' in path: continue
    
    tableA = pd.read_csv('data/' + path  +'/tableA.csv')
    tableB = pd.read_csv('data/' + path  +'/tableB.csv')
    golden = pd.read_csv('data/' + path  +'/matches.csv')
    
    print(path, tableA.shape[0], tableB.shape[0], round((tableA.shape[0] * tableB.shape[0])/1000,1), golden.shape[0])

print()

for task in [
    'Walmart-Amazon',
                'Beer',
                'Amazon-Google',
                'Fodors-Zagat',
                'iTunes-Amazon',
                'DBLP-GoogleScholar',
                'Febrl',
                'DBLP-ACM'
                ]:




    
    folder_root = "data/" + task
    left_df = pd.read_csv(folder_root +'/tableA.csv')
    right_df = pd.read_csv(folder_root +'/tableB.csv')
    match_df = pd.read_csv(folder_root +'/matches.csv')
    match_df.rename(columns={list(match_df.columns)[0]: 'ltable_id', list(match_df.columns)[1]: 'rtable_id'}, inplace=True)



    left_sens = make_sens_vector_(left_df.copy(), task, sens_dict)
    right_sens = make_sens_vector_(right_df.copy(), task, sens_dict)

    
    df_A = pd.DataFrame(left_sens.astype(bool))
    df_B = pd.DataFrame(right_sens.astype(bool))

    match_sens = make_candid_sens(match_df, left_df.copy(), right_df.copy(),task, sens_dict)


    print(task , 'A',np.sum(df_A)[0], 'B',np.sum(df_B)[0], np.sum(match_sens['sensitive']))
    break


In [ ]:

for path in os.listdir('data/'):
    if 'DS_Store' in path: continue
    
    tableA = pd.read_csv('data/' + path  +'/tableA.csv')

    print(path,tableA.shape[1] -1 )


##  Blocking stats, Bias



In [ ]:
import os
import time
import copy
from func import *
import numpy as np
import pandas as pd
from pyjedai.datamodel import Data
from pyjedai.block_cleaning import BlockPurging, BlockFiltering
from pyjedai.comparison_cleaning import CardinalityEdgePruning,BLAST
from pyjedai.block_building import (
    StandardBlocking,
    ExtendedQGramsBlocking,
    ExtendedSuffixArraysBlocking,
    QGramsBlocking,
    SuffixArraysBlocking
)



# Define method dictionaries
classic_method_dict = {
                'SB': StandardBlocking(),
                'QG': QGramsBlocking(),
                'EQG': ExtendedQGramsBlocking(),
                'SA': SuffixArraysBlocking(),
                'ESA': ExtendedSuffixArraysBlocking()
                }
classic_method_name = {
    'SB': 'StandardBlocking',
    'EQG': 'ExtendedQGramsBlocking',
    'ESA': 'ExtendedSuffixArraysBlocking',
    'QG': 'QGramsBlocking',
    'SA': 'SuffixArraysBlocking',
    'CTT': 'CTT',
    'AE': 'AUTO',

}

output_file = "block_stat3_.txt"
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
# 'Fodors-Zagat', 'Walmart-Amazon', 'Amazon-Google','iTunes-Amazon']

tasks = ['DBLP-GoogleScholar' ,'DBLP-ACM','Beer','Fodors-Zagat', 'Walmart-Amazon', 'Amazon-Google','iTunes-Amazon']



tasks = ['iTunes-Amazon','Fodors-Zagat', 'Walmart-Amazon', 'Amazon-Google']
methods = ['SB', 'EQG', 'ESA', 'QG','SA','CTT','AE']


# Process each task with each method
for task in tasks:
    for method in methods:
        if task =='iTunes-Amazon' and method in ['SB', 'EQG', 'ESA', 'QG','SA']: continue
        
        # Load datasets
        left_df = pd.read_csv(f"data/{task}/tableA.csv")
        right_df = pd.read_csv(f"data/{task}/tableB.csv")
        match_df = pd.read_csv(f"data/{task}/matches.csv")


        left_df = left_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df = right_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        
        left_df = left_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        right_df = right_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)



        # Process using classic method if applicable
        if method in classic_method_dict:
            bb = classic_method_dict[method]
            attr = [col for col in left_df.columns if col != 'id']
            data = Data(
                dataset_1=left_df.copy(), id_column_name_1='id',
                dataset_2=right_df.copy(), id_column_name_2='id',
                ground_truth=match_df.rename(columns={list(match_df.columns)[0]: 'D1', list(match_df.columns)[1]: 'D2'}))
            data.clean_dataset(remove_stopwords=False, remove_punctuation=False, remove_numbers=False, remove_unicodes=True)
            
            start_time = time.time()
            blocks = bb.build_blocks(copy.deepcopy(data), attributes_1=attr, attributes_2=attr, tqdm_disable=True)
            
            bp = BlockPurging()
            bf = BlockFiltering()
            mb = BLAST('EJS')
            if task == 'iTunes-Amazon': mb = CardinalityEdgePruning()
            # for meta in META:
            cleaned_blocks = bf.process(copy.deepcopy(blocks), data, tqdm_disable=True)
            filtered_blocks = bp.process(cleaned_blocks, data, tqdm_disable=True)
            candidate_pairs_blocks = mb.process(filtered_blocks, data, tqdm_disable=False)
            candidates = mb.export_to_df(candidate_pairs_blocks)
        
        else:
            if method == 'CTT':
                start_time = time.time()
                candidates = deepBlock(left_df.copy(), right_df.copy(), K = 50, method = 'CTT')
            elif method =='AE':
                start_time = time.time()
                candidates = deepBlock(left_df.copy(), right_df.copy(), K = 50, method = 'AE')
            else:
                print('method not found!')

        end_time = time.time()
        runtime = end_time - start_time

        # Compile result rows
        candidates= candidates.astype(int)
        candidates.rename(columns={'ltable_id': 'id1', 'rtable_id': 'id2'}, inplace=True)

        # Merge left_df and right_df with candidates based on ids using vectorized operations
        left_merged = candidates.merge(left_df, left_on='id1', right_on='id', suffixes=('', '_left'))
        right_merged = left_merged.merge(right_df, left_on='id2', right_on='id', suffixes=('_left', '_right'))

        # Drop redundant columns and reset index
        result_df = right_merged.drop(columns=['id1', 'id2', 'id_left', 'id_right'])
        result_df = right_merged.copy()

        # Merge with match_df to determine labels
        merged_df = result_df.merge(match_df, left_on=['id_left', 'id_right'], right_on=['ltable_id', 'rtable_id'], how='left', indicator=True)
        result_df['label'] = (merged_df['_merge'] == 'both').astype(int)

        # Calculate metrics
        RR = 1 - result_df.shape[0] / (left_df.shape[0] * right_df.shape[0])
        PC = np.sum(result_df['label'] == 1) / match_df.shape[0]
        PQ = np.sum(result_df['label'] == 1) / result_df.shape[0]
        F = 2 * PC * RR / (PC + RR)

        # Print and save results
        print(task, classic_method_name[method])
        print(round(100 * RR, 4), round(100 * PC, 4), round(100 * PQ, 4), round(100 * F, 4))
        print()


        res_sens = []
        if task == 'DBLP-ACM':
            LEFT = pd.DataFrame({'sens':pd.read_csv('data/'+task+'/'+'left_sens.csv')['0'],
            'id':pd.read_csv('data/'+task+'/'+'tableA.csv')['id']})


            RIGHT = pd.DataFrame({'sens':pd.read_csv('data/'+task+'/'+'right_sens.csv')['0'],
            'id':pd.read_csv('data/'+task+'/'+'tableB.csv')['id']})

            a = list(result_df['id1'])
            b = list(result_df['id2'])


            res_sens = []
            for i in range(len(a)):
                sens_c = np.logical_or(list(RIGHT[RIGHT['id'] == b[i]]['sens'])[0],list(LEFT[LEFT['id'] == a[i]]['sens'])[0])
                res_sens.append(sens_c)

        print(1)


        MINOR, MAJOR, data_STAT = calc_bias_block(result_df,match_df,left_df,right_df, task , sens_dict,res_sens )
        [RR_minor,PC_minor,PQ_minor,Fb_minor ] = MINOR
        [RR_major,PC_major,PQ_major,Fb_major ] = MAJOR
        [P_major, P_minor, M_major, M_minor] = data_STAT

        NUM = 2
        print('Minor: ',end='')
        print(round(RR_minor,NUM),round(PC_minor,NUM),round(PQ_minor,NUM), round(Fb_minor,NUM))
        print('major: ',end='')
        print(round(RR_major,NUM),round(PC_major,NUM),round(PQ_major,NUM), round(Fb_major,NUM))
        print('diff : ',end='')
        print(round(RR_major-RR_minor ,NUM),round(PC_major- PC_minor,NUM),round(PQ_major - PQ_minor,NUM),round(Fb_major - Fb_minor,NUM))


        print()
        with open(output_file, "a") as file:
            file.write(f"{task} {classic_method_name[method]}\n")
            file.write(f"{round(100 * RR,8)} {round(100 * PC,8)} {round(100 * PQ,8)} {round(100 * F,8)} {round(runtime, 2)}\n")
            file.write(f"bias {round( RR_minor,8)} {round( PC_minor,8)} {round( PQ_minor,8)} {round( Fb_minor,8)}\n")
            file.write(f"bias {round( RR_major,8)} {round( PC_major,8)} {round( PQ_major,8)} {round( Fb_major,8)}\n")
            file.write(f"bias {round( RR_major - RR_minor,8)} {round( PC_major - PC_minor,8)} {round( PQ_major - PQ_minor,8)} {round( Fb_major - Fb_minor,8)}\n")
            file.write(f"bias {P_major} {P_minor} {M_major} {M_minor}\n\n")







## time experiment

In [ ]:
import os
import time
import copy
import numpy as np
import pandas as pd
from pyjedai.datamodel import Data
from pyjedai.block_cleaning import BlockPurging, BlockFiltering
from pyjedai.comparison_cleaning import CardinalityEdgePruning,BLAST
from pyjedai.block_building import (
    StandardBlocking,
    ExtendedQGramsBlocking,
    ExtendedSuffixArraysBlocking,
    QGramsBlocking,
    SuffixArraysBlocking
)



# Define method dictionaries
classic_method_dict = {
                'SB': StandardBlocking(),
                'QG': QGramsBlocking(),
                'EQG': ExtendedQGramsBlocking(),
                'SA': SuffixArraysBlocking(),
                'ESA': ExtendedSuffixArraysBlocking()
                }
classic_method_name = {
    'SB': 'StandardBlocking',
    'EQG': 'ExtendedQGramsBlocking',
    'ESA': 'ExtendedSuffixArraysBlocking',
    'QG': 'QGramsBlocking',
    'SA': 'SuffixArraysBlocking',

}

output_file = "block_stat_time.txt"
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
tasks = ['Beer','Fodors-Zagat', 'Walmart-Amazon', 'Amazon-Google', 'DBLP-ACM', 'DBLP-GoogleScholar', 'iTunes-Amazon']
methods = ['SB', 'EQG', 'ESA', 'QG','SA']



# Process each task with each method
for task in tasks:
    for method in methods:
        
        time_all = []
        # Load datasets
        left_df = pd.read_csv(f"data/{task}/tableA.csv")
        right_df = pd.read_csv(f"data/{task}/tableB.csv")
        match_df = pd.read_csv(f"data/{task}/matches.csv")


        left_df = left_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df = right_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        
        left_df = left_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        right_df = right_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)

        N= 8
        for _ in range(N):
        # Process using classic method if applicable
            start_time = time.time()

            bb = classic_method_dict[method]
            attr = [col for col in left_df.columns if col != 'id']
            data = Data(
                dataset_1=left_df.copy(), id_column_name_1='id',
                dataset_2=right_df.copy(), id_column_name_2='id',
             ground_truth=match_df.rename(columns={list(match_df.columns)[0]: 'D1', list(match_df.columns)[1]: 'D2'}))
            data.clean_dataset(remove_stopwords=False, remove_punctuation=False, remove_numbers=False, remove_unicodes=True)
            
            
            blocks = bb.build_blocks(copy.deepcopy(data), attributes_1=attr, attributes_2=attr, tqdm_disable=True)
            
            bp = BlockPurging()
            bf = BlockFiltering()
            mb = BLAST('EJS')
            if task == 'iTunes-Amazon': mb = CardinalityEdgePruning()
            # for meta in META:
            cleaned_blocks = bf.process(copy.deepcopy(blocks), data, tqdm_disable=True)
            filtered_blocks = bp.process(cleaned_blocks, data, tqdm_disable=True)
            candidate_pairs_blocks = mb.process(filtered_blocks, data, tqdm_disable=False)
            candidates = mb.export_to_df(candidate_pairs_blocks)
        


            end_time = time.time()
            runtime = end_time - start_time
        
            time_all.append(runtime)
        
        print()
        with open(output_file, "a") as file:
            file.write(f"{task} {classic_method_name[method]}\n")
            file.write(f"{round(np.average(time_all), 4)} {round(np.std(time_all), 4)}\n")






## time for same dataset

In [ ]:
import os
import time
import copy
import numpy as np
import pandas as pd
from pyjedai.datamodel import Data
from pyjedai.block_cleaning import BlockPurging, BlockFiltering
from pyjedai.comparison_cleaning import CardinalityEdgePruning,BLAST
from pyjedai.block_building import (
    StandardBlocking,
    ExtendedQGramsBlocking,
    ExtendedSuffixArraysBlocking,
    QGramsBlocking,
    SuffixArraysBlocking
)



# Define method dictionaries
classic_method_dict = {
                'SB': StandardBlocking(),
                'QG': QGramsBlocking(),
                'EQG': ExtendedQGramsBlocking(),
                'SA': SuffixArraysBlocking(),
                'ESA': ExtendedSuffixArraysBlocking()
                }
classic_method_name = {
    'SB': 'StandardBlocking',
    'EQG': 'ExtendedQGramsBlocking',
    'ESA': 'ExtendedSuffixArraysBlocking',
    'QG': 'QGramsBlocking',
    'SA': 'SuffixArraysBlocking',

}

output_file = "block_stat_time_same5_.txt"
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
tasks = ['Walmart-Amazon']
methods = ['SB', 'EQG', 'ESA', 'QG','SA']



# Process each task with each method
for task in tasks:
    for method in methods:
        
        time_all = []
        # Load datasets
        left_df_orig = pd.read_csv(f"data/{task}/tableA.csv")
        right_df_orig = pd.read_csv(f"data/{task}/tableB.csv")
        match_df_orig = pd.read_csv(f"data/{task}/matches.csv")
        




        left_df_orig = left_df_orig.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df_orig = right_df_orig.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        
        left_df_orig = left_df_orig.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        right_df_orig = right_df_orig.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        SIZE = left_df_orig.shape[0] * right_df_orig.shape[0]

        for f in [0.01, 0.2,0.4,0.6,0.8,1]:
        # for f in [0.01,0.5,1]:
            F = np.sqrt(f)
            left_df = copy.deepcopy(left_df_orig).sample(frac =F,replace=False)
            right_df = copy.deepcopy(right_df_orig).sample(frac =F,replace=False)
            match_df = copy.deepcopy(match_df_orig)
            match_df = match_df[match_df['ltable_id'].isin(list(left_df['id']))]
            match_df = match_df[match_df['rtable_id'].isin(list(right_df['id']))]

        # Process using classic method if applicable
            runtime = 0
            for _ in range(1):
                start_time = time.time()

                bb = classic_method_dict[method]
                attr = [col for col in left_df.columns if col != 'id']
                data = Data(
                    dataset_1=left_df.copy(), id_column_name_1='id',
                    dataset_2=right_df.copy(), id_column_name_2='id',
                ground_truth=match_df.rename(columns={list(match_df.columns)[0]: 'D1', list(match_df.columns)[1]: 'D2'}))
                data.clean_dataset(remove_stopwords=False, remove_punctuation=False, remove_numbers=False, remove_unicodes=True)
                
                
                blocks = bb.build_blocks(copy.deepcopy(data), attributes_1=attr, attributes_2=attr, tqdm_disable=True)
                
                bp = BlockPurging()
                bf = BlockFiltering()
                mb = BLAST('EJS')
                # for meta in META:
                cleaned_blocks = bf.process(copy.deepcopy(blocks), data, tqdm_disable=True)
                filtered_blocks = bp.process(cleaned_blocks, data, tqdm_disable=True)
                candidate_pairs_blocks = mb.process(filtered_blocks, data, tqdm_disable=False)
                candidates = mb.export_to_df(candidate_pairs_blocks)
            


                end_time = time.time()
                runtime+= end_time - start_time
                
            runtime = runtime /1   
            size_ = left_df.shape[0] * right_df.shape[0]
            print()
            with open(output_file, "a") as file:
                if f == 0.01:
                    file.write(f"{task} {classic_method_name[method]}\n")
                file.write(f"{round(F**2,2)} {size_},{SIZE} {round(np.average(runtime), 4)}\n")
                if f == 1:
                    file.write(f"\n")
            

            





## time for deep


In [ ]:
import os
import time
import copy
import numpy as np
import pandas as pd
from func import *



output_file = "block_deep_stat_time.txt"
try: os.remove(output_file)
except FileNotFoundError: pass

# Define tasks and methods to iterate over
tasks = ['Beer','Fodors-Zagat', 'Walmart-Amazon', 'Amazon-Google', 'DBLP-ACM', 'DBLP-GoogleScholar', 'iTunes-Amazon']



# Process each task with each method
for method in ['AE','CTT']:
    for task in tasks:
    
        
        left_df = pd.read_csv(f"data/{task}/tableA.csv")
        right_df = pd.read_csv(f"data/{task}/tableB.csv")

        left_df = left_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df = right_df.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        
        left_df = left_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        right_df = right_df.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)

        print(task , method)
        # start_time = time.time()
        candidates = deepBlock(left_df.copy(), right_df.copy(), K = 50, method = method)
        # end_time = time.time()
        # runtime = end_time - start_time
        
        
        print()
        # with open(output_file, "a") as file:
        #     file.write(f"{task} {method}\n")
        #     file.write(f"{round(np.average(time_all), 4)} {round(np.std(time_all), 4)}\n")






## time for same dataset deep


In [1]:
import os
import time
import copy
import numpy as np
import pandas as pd
from func import *

# Define tasks and methods to iterate over
tasks = ['Walmart-Amazon']




# Process each task with each method
for method in ['AE','CTT']:
    for task in tasks:
        
        left_df_orig = pd.read_csv(f"data/{task}/tableA.csv")
        right_df_orig = pd.read_csv(f"data/{task}/tableB.csv")
        
        left_df_orig = left_df_orig.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        right_df_orig = right_df_orig.replace(r"\\ '", "'", regex=True).replace(r" '", "'", regex=True).replace(r"\\ `", "\\ ", regex=True)
        
        left_df_orig = left_df_orig.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        right_df_orig = right_df_orig.applymap(lambda x: x.strip('`') if isinstance(x, str) else x).applymap(lambda x: x.strip("'") if isinstance(x, str) else x).applymap(lambda x: x.strip() if isinstance(x, str) else x)
        SIZE = left_df_orig.shape[0] * right_df_orig.shape[0]

        # break
        for f in [0.01, 0.2,0.4,0.6,0.8,1]:
            F = np.sqrt(f)
            left_df = copy.deepcopy(left_df_orig).sample(frac =F,replace=False)
            right_df = copy.deepcopy(right_df_orig).sample(frac =F,replace=False)
            left_df = left_df.fillna('NA')
            right_df = right_df.fillna('NA')



            print(task , method, f)
            candidates = deepBlock(left_df.copy(), right_df.copy(), K = 50, method = method)
        
        
        print()
        print()






Walmart-Amazon AE 0.01


2.740363121032715
2.242300033569336
2.364861249923706
2.329403877258301
2.12947678565979
Walmart-Amazon AE 0.2


8.828837871551514
8.53029465675354
7.884539842605591
7.125627040863037
7.668518781661987
Walmart-Amazon AE 0.4


11.403317928314209
10.389057159423828
10.333085060119629
10.438457012176514
10.262450218200684
Walmart-Amazon AE 0.6


13.923197984695435
13.501504182815552
12.9218270778656
13.242563962936401
12.90197491645813
Walmart-Amazon AE 0.8


...

13.641733169555664
9.252118110656738
9.29131007194519
9.17280387878418
8.694905996322632
Walmart-Amazon CTT 0.4


17.919050931930542
13.901690244674683
14.308176040649414
14.26353907585144
13.692553997039795
Walmart-Amazon CTT 0.6


29.114394187927246
19.582361936569214
17.507019996643066
18.043914794921875
17.371819972991943
Walmart-Amazon CTT 0.8


26.47039794921875
21.09325909614563
20.620249032974243
20.82024312019348
20.53084373474121
Walmart-Amazon CTT 1


32.74311709403992
24.53022003173828
23.9623601436615
24.175256967544556
24.379826068878174


